In [1]:
!pip install gurobipy

In [2]:
from gurobipy import *

## 問題例2

In [3]:
S=[i for i in range(1,5)]
H=[i for i in range(1,4)]
P=[i for i in range(1,9)]
T=[i for i in range(1,5)]
n=[1 for i in range(1,9)]


In [4]:
m=Model("IP3")

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
x,y={},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")

for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="C",lb=0)



In [7]:
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

In [8]:
for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])

In [9]:
for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[s,k,t+1] for s in S if s!=k)<=1)

In [10]:
m.addConstr(x[1,1,6,1]==1)
m.addConstr(x[1,2,5,1]==1)
m.addConstr(x[2,1,1,1]==1)
m.addConstr(x[2,2,7,1]==1)
m.addConstr(x[3,1,3,1]==1)
m.addConstr(x[3,2,4,1]==1)
m.addConstr(x[3,3,2,1]==1)
m.addConstr(x[4,1,8,1]==1)


<gurobi.Constr *Awaiting Model Update*>

In [23]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T))))
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 913 rows, 420 columns and 6252 nonzeros
Model fingerprint: 0x04511dba
Variable types: 36 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 3 (0.02s)
Loaded MIP start from previous solve with objective 3

Presolve removed 432 rows and 191 columns
Presolve time: 0.05s
Presolved: 481 rows, 229 columns, 3104 nonzeros
Variable types: 36 continuous, 193 integer (193 binary)

Root relaxation: objective 2.142857e+00, 216 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.14286 

In [25]:
EPS=1.e-6

for (s,h,p,t) in x:
  if t==1:
    if x[s,h,p,t].X>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  if t>=2:
    if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

for (s,k,t) in y:
  if y[s,k,t].X>EPS:
    print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))
print("the number of moves", m.objVal)

x[ 1, 1, 6, 1]=1.0
x[ 1, 2, 5, 1]=1.0
x[ 1, 3, 2, 3]=1.0
x[ 2, 1, 1, 1]=1.0
x[ 2, 2, 7, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 4, 1]=1.0
x[ 3, 3, 2, 1]=1.0
x[ 4, 1, 8, 1]=1.0
x[ 4, 2, 7, 2]=1.0
x[ 4, 3, 4, 4]=1.0
y[ 2, 4, 1]=1.0
y[ 3, 1, 2]=1.0
y[ 3, 4, 3]=1.0
the number of moves 3.0
